# First you may check the list of options by Minimac4

In [ ]:
%matplotlib inline

In [ ]:
cp -r /home/jupyter-bgi23_siyang/software/Minimac4 .
cd ./Minimac4
mkdir -p ./test/output
minimac4=`pwd`/build/minimac4 
input=`pwd`/test/input
output=`pwd`/test/output
$minimac4 --help


# Format the data

In [ ]:
bcftools view ${input}/ref_panel_with_haploid_sample.vcf -Oz -o ${output}/ref_panel_with_haploid_sample.vcf.gz
bcftools view ${input}/tar_panel_with_haploid_sample.vcf -Oz -o ${output}/tar_panel_with_haploid_sample.vcf.gz

bcftools index ${output}/ref_panel_with_haploid_sample.vcf.gz
bcftools index ${output}/tar_panel_with_haploid_sample.vcf.gz
less -S ${input}/ref_panel_with_haploid_sample.vcf|wc -l

# Reference Panel Creation

In [ ]:
$minimac4 --compress-reference \
    ${output}/ref_panel_with_haploid_sample.vcf.gz > ${output}/ref_panel_with_haploid_sample.msav


# A typical Minimac4 command line for imputation

In [ ]:
$minimac4 ${output}/ref_panel_with_haploid_sample.msav ${output}/tar_panel_with_haploid_sample.vcf.gz -f GT -O vcf.gz --temp-buffer 2 > ${output}/imputed_vcf.gz



# View the result

In [ ]:
less -S ${output}/imputed_vcf.gz|head -n 20
gunzip -c ${output}/imputed_vcf.gz|wc -l

In [ ]:
drawr2=/home/jupyter-bgi23_siyang/scripts/draw_r2.py
python $drawr2 ${output}/imputed_vcf.gz

In [ ]:
#Visualize distribution of the MACH R2 
import sys
import gzip
import re
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


if __name__ == "__main__":
    in_fname = "Minimac4/test/output/imputed_vcf.gz"

    data = []
    with gzip.open(in_fname, "rt") if in_fname.endswith(".gz") else open(in_fname) as f:
        for line in f:
            if line.startswith("#"):
                continue

            col = line.strip().split()

            # af = float(re.search(";?AF=([^;]+)", col[7]).group(1))
            m = re.search(";?R2=([^;]+)", col[7])
            if m:
                r2 = float(m.group(1))
                data.append(r2)


    df = pd.DataFrame({"R2": data})

    f, ax = plt.subplots(1,1,figsize=(6, 4), constrained_layout=True)
    sns.histplot(data=df[df["R2"]!=0], x="R2", kde=True, color="b", ax=ax)
    ax.set_yscale("log")
    plt.show()
